In [ ]:
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt


from scipy.ndimage import label, center_of_mass
from skimage.draw import disk
from skimage.transform import radon, iradon, resize
import cv2
import imageio.v2 as iio
from skimage import util
import annotator
import h5py

%matplotlib widget

In [ ]:
def read_h5_file(file_path):
    with h5py.File(file_path, 'r') as f:
        # print("Keys: %s" % f.keys())
        data = f['data'][:]
    return data

def create_noisy_image(image, num_projection=1000, rotation = 360, undersample_factor=4, noise_level=2):
    image = image.astype(float)
    image = (image - np.min(image)) / (np.max(image) - np.min(image))
    theta = np.linspace(0., rotation, int(num_projection/undersample_factor), endpoint=False)
    sinogram = radon(image, theta=theta)
    sinogram += noise_level * np.random.normal(size=sinogram.shape)
    reconstruction_fbp = iradon(sinogram, theta=theta)
    error = reconstruction_fbp - image
    print(f'FBP rms reconstruction error: {np.sqrt(np.mean(error**2)):.3g}')
    return reconstruction_fbp, error, sinogram

def plot_sinogram_recon(sinogram, reconstruction_fbp, error, num_projection, undersample_factor, rotation):
    dx, dy = 0.5 * rotation / int(num_projection/undersample_factor), 0.5 / sinogram.shape[0]
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.set_title("Radon transform\n(Sinogram)")
    ax.set_xlabel("Projection angle (deg)")
    ax.set_ylabel("Projection position (pixels)")
    ax.imshow(sinogram, cmap=plt.cm.Greys_r,
            extent=(-dx, rotation + dx, -dy, sinogram.shape[0] + dy),
            aspect='auto')
    fig.tight_layout()
    plt.show()

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5),
                                sharex=True, sharey=True)
    ax1.set_title("Reconstruction\nFiltered back projection")
    ax1.imshow(reconstruction_fbp, cmap=plt.cm.Greys_r)
    ax2.set_title("Reconstruction error\nFiltered back projection")
    ax2.imshow(error, cmap=plt.cm.Greys_r)
    plt.show()

def make_circular_image(image):
    center_x = image.shape[1] // 2
    center_y = image.shape[0] // 2
    radius = min(image.shape) // 2
    circle_image = np.copy(image)
    cv2.circle(circle_image, (center_x, center_y), radius, (0, 0, 255), 2)
    mask = np.zeros_like(image)
    cv2.circle(mask, (center_x, center_y), radius, (255, 255, 255), -1)
    circle_image[mask == 0] = 0
    return circle_image

def save_mask(file_path, new_file_name):
    mask = np.array(Image.open(file_path))[:,:,0]/255
    mask = util.invert(mask)
    save_mask = (mask * 255).astype(np.uint8)
    iio.imwrite(new_file_name, save_mask)

def normalize_uint8(image):
    image = (image - np.min(image)) / (np.max(image) - np.min(image))
    return (image * 255).astype(np.uint8)

def create_mask_with_centroids(mask, radius=3):
    # New curated mask with radius 3 around the centroids of the mask
    labeled_mask, num_features = label(mask)
    centroids = center_of_mass(mask, labeled_mask, range(1, num_features+1))
    print(f'true_centroids: {len(centroids)}')
    new_mask = np.zeros_like(mask)
    for y, x in centroids:
        rr, cc = disk((y, x), radius)
        new_mask[rr, cc] = 1
    return new_mask

### Mock data preparation

#### Mask preparation

In [ ]:
# Save mock and UD slice 500 in uint8 tiff file format

im = np.array(Image.open('final_data/ScanRef_Glass_Mock_UD/im_Mock500.tiff'))
# im = make_circular_image(im) # Make the image square (988x988) - original size (988x1013)
# iio.imwrite('final_data/ScanRef_Glass_Mock_UD/im_Mock500.tiff', im) # save the reference image

# convert to 8 bit and annotate it if mask is not available
im = (im - np.min(im)) / (np.max(im) - np.min(im)) * 255
im_uint8 = im.astype(np.uint8)
# annotator.annotate(im_uint8, 'Mock500_mask.png')

In [ ]:
# save mask in binary format
file_name = 'Mock500_mask.png'
file_path = os.path.join('final_data/ScanRef_Glass_Mock_UD/', file_name)
new_file_name = 'mask_Mock500.png'
# save_mask(file_path, new_file_name)

#### noisy data

In [ ]:
mask = np.array(Image.open('final_data/ScanRef_Glass_Mock_UD/mask_Mock500.png'))
saving_folder = 'final_data/ScanRef_Glass_Mock_UD/'
new_mask_name = 'curatedmask_Mock500.png'


# create new curated mask with centroids
# new_mask = create_mask_with_centroids(mask, radius=3)
# plt.figure()
# plt.imshow(new_mask, cmap='gray')
# plt.imsave(os.path.join(saving_folder, new_mask_name), new_mask, cmap='gray')

undersample_factor = [8, 12]
noise_level = [4, 8, 12, 16, 20, 24]
proj = 4501
rot = 360

for i in range(len(undersample_factor)):
    for j in range(len(noise_level)):
        print(f'undersample_factor: {undersample_factor[i]}, noise_level: {noise_level[j]}')
        reconstruction_fbp, error, sinogram = create_noisy_image(im, num_projection=proj, rotation = rot, undersample_factor=undersample_factor[i], noise_level=noise_level[j])
        plot_sinogram_recon(sinogram, reconstruction_fbp, error, num_projection=proj, undersample_factor=undersample_factor[i], rotation=rot)
        file_name = f'im_Mock500noisy_uf{undersample_factor[i]}_n{noise_level[j]}.tiff'
        reconstruction_fbp = normalize_uint8(reconstruction_fbp)
        # iio.imwrite(os.path.join(saving_folder, str(file_name)), reconstruction_fbp)

### T700 data preparation

#### Mask preparation

In [ ]:
data_name = ['T700-T-02_pco_0p4um.h5', 'T700-T-08_pco_0p4um.h5', 'T700-T-21_GF_0p8um_1ms_1.h5', 
             'T700-T-21_GF_0p8um_1ms_2.h5', 'T700-T-21_GF_0p8um_1ms_3.h5', 'T700-T-21_GF_1p6um_0p5ms_1.h5',
             'T700-T-21_GF_1p6um_0p5ms_2.h5', 'T700-T-21_GF_1p6um_3ms_1.h5', 'T700-T-21_GF_1p6um_3ms_2.h5',
             'T700-T-21_GF_1p6um_3ms_3.h5', 'T700-T-21_GF_1p6um_3ms_4.h5', 'T700-T-21_GF_1p6um_3ms_5.h5',
             'T700-T-21_pco_0p4um_reference.h5', 'T700-T-26_pco_0p4um.h5']
data = read_h5_file('../Data_CF/' + data_name[12])
# data = read_h5_file('../Data_CF/' + 'T700-T-21_pco_0p4um_reference.h5')
im1 = np.array(data)
im = np.array(data[int(len(data)//4),  500:1100, 42:]) # 361 slice of the data [600x600] square image of reference data
im = im.max() - im # actual data is inverted
im = (im - np.min(im)) / (np.max(im) - np.min(im)) * 255
im = im.astype(np.uint8)
print(f'slice number: {int(len(data)//4)}')

In [ ]:
im = make_circular_image(im) # make the image circular to use fbp reconstruction with various noise level effectivly
print(im.max())
plt.figure()
plt.imshow(im, cmap='gray')
plt.title(data_name[12])
plt.show()

In [ ]:
# iio.imwrite('T700_ref_361.tiff', im) # save the reference image

In [ ]:
# convert to 8 bit and annotate it if mask is not available
im = (im - np.min(im)) / (np.max(im) - np.min(im)) * 255
im_uint8 = im.astype(np.uint8)
# annotator.annotate(im_uint8, 'T700_ref_mask_361.png')

In [ ]:
# save mask
file_name = 'T700_ref_mask_361.png'
file_path = os.path.join(os.getcwd(), file_name)
new_file_name = 'mask_T700_ref_361.png'
# save_mask(file_path, new_file_name)

#### noisy data preparation

In [ ]:
im = np.array(Image.open('final_data/T700/im_T700ref361.tiff'))
mask = np.array(Image.open('final_data/T700/mask_T700ref361.png'))
saving_folder = 'final_data/T700/'
new_mask_name = 'curatedmask_T700ref361.png'

# create mask with centroids
# new_mask = create_mask_with_centroids(mask, radius=3)
# plt.figure()
# plt.imshow(new_mask, cmap='gray')
# plt.imsave(os.path.join(saving_folder, new_mask_name), new_mask, cmap='gray')

undersample_factor = [8, 12]
noise_level = [2, 3, 4, 5, 6, 7]
proj = 2000
rot = 180

for i in range(len(undersample_factor)):
    for j in range(len(noise_level)):
        print(f'undersample_factor: {undersample_factor[i]}, noise_level: {noise_level[j]}')
        reconstruction_fbp, error, sinogram = create_noisy_image(im, num_projection=proj, rotation = rot, undersample_factor=undersample_factor[i], noise_level=noise_level[j])
        plot_sinogram_recon(sinogram, reconstruction_fbp, error, num_projection=proj, undersample_factor=undersample_factor[i], rotation=rot)
        file_name = f'im_T700ref361noisy_uf{undersample_factor[i]}_n{noise_level[j]}.tiff'
        reconstruction_fbp = normalize_uint8(reconstruction_fbp)
        # iio.imwrite(os.path.join(saving_folder, str(file_name)), reconstruction_fbp)